In [3]:
"""
Run the homography adaptation for all images in a given folder
to regress and aggregate line distance function maps.
"""

import os
import argparse
import numpy as np
import cv2
import h5py
import torch
from tqdm import tqdm
from pytlsd import lsd
from afm_op import afm
from joblib import Parallel, delayed

from ..datasets.utils.homographies import sample_homography, warp_lines
from ..datasets.utils.data_augmentation import random_contrast


homography_params = {
}


def ha_df(img, num=100, border_margin=3, min_counts=5):
    """ Perform homography adaptation to regress line distance function maps.
    Args:
        img: a grayscale np image.
        num: number of homographies used during HA.
        border_margin: margin used to erode the boundaries of the mask.
        min_counts: any pixel which is not activated by more than min_count is BG.
    Returns:
        The aggregated distance function maps in pixels
        and the angle to the closest line.
    """
    h, w = img.shape[:2]
    size = (w, h)
    df_maps, angles, closests, counts = [], [], [], []
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,
                                       (border_margin * 2, border_margin * 2))
    pix_loc = np.stack(np.meshgrid(np.arange(h), np.arange(w), indexing='ij'),
                       axis=-1)
    raster_lines = np.zeros_like(img)

    # Loop through all the homographies
    for i in range(num):
        # Generate a random homography
        if i == 0:
            H = np.eye(3)
        else:
            H = sample_homography(img.shape, **homography_params)
        H_inv = np.linalg.inv(H)
        
        # Warp the image
        warped_img = cv2.warpPerspective(img, H, size,
                                         borderMode=cv2.BORDER_REPLICATE)
        
        # Regress the DF on the warped image
        warped_lines = lsd(warped_img)[:, [1, 0, 3, 2]].reshape(-1, 2, 2)
        
        # Warp the lines back
        lines = warp_lines(warped_lines, H_inv)
        
        # Get the DF and angles
        num_lines = len(lines)
        cuda_lines = torch.from_numpy(lines[:, :, [1, 0]].astype(np.float32))
        cuda_lines = cuda_lines.reshape(-1, 4)[None].cuda()
        offset = afm(
            cuda_lines,
            torch.IntTensor([[0, num_lines, h, w]]).cuda(), h, w)[0]
        offset = offset[0].permute(1, 2, 0).cpu().numpy()[:, :, [1, 0]]
        closest = pix_loc + offset
        df = np.linalg.norm(offset, axis=-1)
        angle = np.mod(np.arctan2(
            offset[:, :, 0], offset[:, :, 1]) + np.pi / 2, np.pi)
        
        df_maps.append(df)
        angles.append(angle)
        closests.append(closest)
        
        # Compute the valid pixels
        count = cv2.warpPerspective(np.ones_like(img), H_inv, size,
                                    flags=cv2.INTER_NEAREST)
        count = cv2.erode(count, kernel)
        counts.append(count)
        raster_lines += (df < 1).astype(np.uint8) * count 
        
    # Compute the median of all DF maps, with counts as weights
    df_maps, angles = np.stack(df_maps), np.stack(angles)
    counts, closests = np.stack(counts), np.stack(closests)
    
    # Median of the DF
    df_maps[counts == 0] = np.nan
    avg_df = np.nanmedian(df_maps, axis=0)

    # Median of the closest
    closests[counts == 0] = np.nan
    avg_closest = np.nanmedian(closests, axis=0)

    # Median of the angle
    circ_bound = (np.minimum(np.pi - angles, angles)
                  * counts).sum(0) / counts.sum(0) < 0.3
    angles[:, circ_bound] -= np.where(
        angles[:, circ_bound] > np.pi /2,
        np.ones_like(angles[:, circ_bound]) * np.pi,
        np.zeros_like(angles[:, circ_bound]))
    angles[counts == 0] = np.nan
    avg_angle = np.mod(np.nanmedian(angles, axis=0), np.pi)

    # Generate the background mask and a saliency score
    raster_lines = np.where(raster_lines > min_counts, np.ones_like(img),
                            np.zeros_like(img))
    raster_lines = cv2.dilate(raster_lines, np.ones((21, 21), dtype=np.uint8))
    bg_mask = (1 - raster_lines).astype(float)

    return avg_df, avg_angle, avg_closest[:, :, [1, 0]], bg_mask


def process_image(img_path, randomize_contrast, num_H, output_folder):
    img = cv2.imread(img_path, 0)
    if randomize_contrast is not None:
        img = randomize_contrast(img)
    
    # Run homography adaptation
    df, angle, closest, bg_mask = ha_df(img, num=num_H)

    # Save the DF in a hdf5 file
    out_path = os.path.splitext(os.path.basename(img_path))[0]
    out_path = os.path.join(output_folder, out_path) + '.hdf5'
    with h5py.File(out_path, "w") as f:
        f.create_dataset("df", data=df.flatten())
        f.create_dataset("line_level", data=angle.flatten())
        f.create_dataset("closest", data=closest.flatten())
        f.create_dataset("bg_mask", data=bg_mask.flatten())


def export_ha(images_list, output_folder, num_H=100,
              rdm_contrast=False, n_jobs=1):
    # Parse the data
    with open(images_list, 'r') as f:
        image_files = f.readlines()
    image_files = [path.strip('\n') for path in image_files]
    
    # Random contrast object
    randomize_contrast = random_contrast() if rdm_contrast else None
    
    # Process each image in parallel
    Parallel(n_jobs=n_jobs, backend='multiprocessing')(delayed(process_image)(
        img_path, randomize_contrast, num_H, output_folder)
                                            for img_path in tqdm(image_files))


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('images_list', type=str,
                        help='Path to a txt file containing the image paths.')
    parser.add_argument('output_folder', type=str, help='Output folder.')
    parser.add_argument('--num_H', type=int, default=100,
                        help='Number of homographies used during HA.')
    parser.add_argument('--random_contrast', action='store_true',
                        help='Add random contrast to the images (disabled by default).')
    parser.add_argument('--n_jobs', type=int, default=1,
                        help='Number of jobs to run in parallel.')
    args = parser.parse_args()

    export_ha(args.images_list, args.output_folder, args.num_H,
              args.random_contrast, args.n_jobs)

ModuleNotFoundError: No module named 'afm_op'

In [ ]:
"""
2D visualization primitives based on Matplotlib.

1) Plot images with `plot_images`.
2) Call `plot_keypoints` or `plot_matches` any number of times.
3) Optionally: save a .png or .pdf plot (nice in papers!) with `save_plot`.
"""

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import flow_vis


def cm_RdGn(x):
    """Custom colormap: red (0) -> yellow (0.5) -> green (1)."""
    x = np.clip(x, 0, 1)[..., None]*2
    c = x*np.array([[0, 1., 0]]) + (2-x)*np.array([[1., 0, 0]])
    return np.clip(c, 0, 1)


def plot_images(imgs, titles=None, cmaps='gray', dpi=100, size=6, pad=.5):
    """Plot a set of images horizontally.
    Args:
        imgs: a list of NumPy or PyTorch images, RGB (H, W, 3) or mono (H, W).
        titles: a list of strings, as titles for each image.
        cmaps: colormaps for monochrome images.
    """
    n = len(imgs)
    if not isinstance(cmaps, (list, tuple)):
        cmaps = [cmaps] * n
    figsize = (size*n, size*3/4) if size is not None else None
    fig, ax = plt.subplots(1, n, figsize=figsize, dpi=dpi)
    if n == 1:
        ax = [ax]
    for i in range(n):
        ax[i].imshow(imgs[i], cmap=plt.get_cmap(cmaps[i]))
        ax[i].get_yaxis().set_ticks([])
        ax[i].get_xaxis().set_ticks([])
        ax[i].set_axis_off()
        for spine in ax[i].spines.values():  # remove frame
            spine.set_visible(False)
        if titles:
            ax[i].set_title(titles[i])
    fig.tight_layout(pad=pad)
        
        
def plot_lines(lines, line_colors='orange', point_color='cyan',
               ps=4, lw=2, indices=(0, 1), alpha=1):
    """ Plot lines and endpoints for existing images.
    Args:
        lines: list of ndarrays of size (N, 2, 2).
        line_colors: string, or list of list of tuples (one for per line).
        point_color: unique color for all endpoints.
        ps: size of the keypoints as float pixels.
        lw: line width as float pixels.
        indices: indices of the images to draw the matches on.
        alpha: alpha transparency.
    """
    if not isinstance(line_colors, list):
        line_colors = [[line_colors] * len(l) for l in lines]
    for i in range(len(lines)):
        if ((not isinstance(line_colors[i], list))
            and (not isinstance(line_colors[i], np.ndarray))):
            line_colors[i] = [line_colors[i]] * len(lines[i])

    fig = plt.gcf()
    ax = fig.axes
    assert len(ax) > max(indices)
    axes = [ax[i] for i in indices]
    fig.canvas.draw()

    # Plot the lines and junctions
    for a, l, lc in zip(axes, lines, line_colors):
        for i in range(len(l)):
            line = matplotlib.lines.Line2D(
                (l[i, 0, 0], l[i, 1, 0]), (l[i, 0, 1], l[i, 1, 1]),
                zorder=1, c=lc[i], linewidth=lw, alpha=alpha)
            a.add_line(line)
        pts = l.reshape(-1, 2)
        a.scatter(pts[:, 0], pts[:, 1], c=point_color, s=ps,
                  linewidths=0, zorder=2, alpha=alpha)

        
def plot_vp(lines, vp_labels, lw=2, indices=(0, 1)):
    """ Plot the vanishing directions of the lines, given the vp labels.
    Lines labelled with -1 are ignored.
    Args:
        lines: list of ndarrays of size (N, 2, 2).
        vp_labels: list of labels indicating the corresponding vp.
        lw: line width as float pixels.
        indices: indices of the images to draw the matches on.
    """
    num_labels = np.amax([np.amax(vp) for vp in vp_labels if len(vp) > 0]) + 1
    colors = sns.color_palette("hls", num_labels)
    
    fig = plt.gcf()
    ax = fig.axes
    assert len(ax) > max(indices)
    axes = [ax[i] for i in indices]
    fig.canvas.draw()

    # Plot the lines and junctions
    for a, l, vp in zip(axes, lines, vp_labels):
        for i in range(len(l)):
            if vp[i] == -1:
                continue
            line = matplotlib.lines.Line2D(
                (l[i, 0, 0], l[i, 1, 0]), (l[i, 0, 1], l[i, 1, 1]),
                zorder=1, c=colors[vp[i]], linewidth=lw)
            a.add_line(line)


def plot_color_line_matches(lines, correct_matches=None,
                            lw=2, indices=(0, 1)):
    """Plot line matches for existing images with multiple colors.
    Args:
        lines: list of ndarrays of size (N, 2, 2).
        correct_matches: bool array of size (N,) indicating correct matches.
        lw: line width as float pixels.
        indices: indices of the images to draw the matches on.
    """
    n_lines = len(lines[0])
    colors = sns.color_palette('husl', n_colors=n_lines)
    np.random.shuffle(colors)
    alphas = np.ones(n_lines)
    # If correct_matches is not None, display wrong matches with a low alpha
    if correct_matches is not None:
        alphas[~np.array(correct_matches)] = 0.2

    fig = plt.gcf()
    ax = fig.axes
    assert len(ax) > max(indices)
    axes = [ax[i] for i in indices]
    fig.canvas.draw()

    # Plot the lines
    for a, l in zip(axes, lines):
        # Transform the points into the figure coordinate system
        transFigure = fig.transFigure.inverted()
        endpoint0 = transFigure.transform(a.transData.transform(l[:, 0]))
        endpoint1 = transFigure.transform(a.transData.transform(l[:, 1]))
        fig.lines += [matplotlib.lines.Line2D(
            (endpoint0[i, 0], endpoint1[i, 0]),
            (endpoint0[i, 1], endpoint1[i, 1]),
            zorder=1, transform=fig.transFigure, c=colors[i],
            alpha=alphas[i], linewidth=lw) for i in range(n_lines)]


def plot_color_lines(lines, correct_matches, wrong_matches,
                     lw=2, indices=(0, 1)):
    """Plot line matches for existing images with multiple colors:
    green for correct matches, red for wrong ones, and blue for the rest.
    Args:
        lines: list of ndarrays of size (N, 2, 2).
        correct_matches: list of bool arrays of size N with correct matches.
        wrong_matches: list of bool arrays of size (N,) with correct matches.
        lw: line width as float pixels.
        indices: indices of the images to draw the matches on.
    """
    # palette = sns.color_palette()
    palette = sns.color_palette("hls", 8)
    blue = palette[5]  # palette[0]
    red = palette[0]  # palette[3]
    green = palette[2]  # palette[2]
    colors = [np.array([blue] * len(l)) for l in lines]
    for i, c in enumerate(colors):
        c[np.array(correct_matches[i])] = green
        c[np.array(wrong_matches[i])] = red

    fig = plt.gcf()
    ax = fig.axes
    assert len(ax) > max(indices)
    axes = [ax[i] for i in indices]
    fig.canvas.draw()

    # Plot the lines
    for a, l, c in zip(axes, lines, colors):
        # Transform the points into the figure coordinate system
        transFigure = fig.transFigure.inverted()
        endpoint0 = transFigure.transform(a.transData.transform(l[:, 0]))
        endpoint1 = transFigure.transform(a.transData.transform(l[:, 1]))
        fig.lines += [matplotlib.lines.Line2D(
            (endpoint0[i, 0], endpoint1[i, 0]),
            (endpoint0[i, 1], endpoint1[i, 1]),
            zorder=1, transform=fig.transFigure, c=c[i],
            linewidth=lw) for i in range(len(l))]


def get_flow_vis(df, ang, line_neighborhood=5):
    norm = line_neighborhood + 1 - np.clip(df, 0, line_neighborhood)
    flow_uv = np.stack([norm * np.cos(ang), norm * np.sin(ang)], axis=-1)
    flow_img = flow_vis.flow_to_color(flow_uv, convert_to_bgr=False)
    return flow_img


def save_plot(path, **kw):
    """Save the current figure without any white margin."""
    plt.savefig(path, bbox_inches='tight', pad_inches=0)
    




In [2]:
MINIDEPTH_PATH = '/cluster/courses/3dv/data/team-2/minidepth/images/'

In [5]:
from PIL import Image

# Load the image
image_path = os.path.join(MINIDEPTH_PATH, '0000/3277894615_328d32e572_o.jpg')
# image_path = '/home/egoedeke/Downloads/MegaDepth_v1/phoenix/S6/zl548/MegaDepth_v1/0377/dense0/imgs/138408872_2f903a81d4_o.jpg'
image = Image.open(image_path)

# Get dimensions
width, height = image.size

print(f"Width: {width}, Height: {height}")


Width: 950, Height: 1280


In [6]:
import h5py

# Open the HDF5 file
file = h5py.File('/home/egoedeke/OneDrive/ETH/3DVision/glue-factory/GT_lines/138408872_2f903a81d4_o.hdf5', 'r')

PermissionError: [Errno 13] Unable to synchronously open file (unable to open file: name = '/home/egoedeke/OneDrive/ETH/3DVision/glue-factory/GT_lines/138408872_2f903a81d4_o.hdf5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)

In [ ]:


print("Keys in file: ", list(file.keys()))

df = file['df'][:]
line_level = file['line_level'][:]
closest = file['closest'][:]
bg_mask = file['bg_mask'][:]

# print dimensions of the data
print("df shape: ", df.shape)
print("line_level shape: ", line_level.shape)
print("closest shape: ", closest.shape)
print("bg_mask shape: ", bg_mask.shape)



Keys in file:  ['bg_mask', 'closest', 'df', 'line_level']
df shape:  (1897600,)
line_level shape:  (1897600,)
closest shape:  (3795200,)
bg_mask shape:  (1897600,)


In [ ]:
import matplotlib.pyplot as plt

# Load the image
image = Image.open('/home/egoedeke/Downloads/MegaDepth_v1/phoenix/S6/zl548/MegaDepth_v1/0377/dense0/imgs/138408872_2f903a81d4_o.jpg')

# Get dimensions
# width, height = image.size

width, height = 1600, df.shape[0] // 1600


# width, height = 640, 480

# display orginal image
#plt.imshow(image)

# Reshape the flattened arrays to the original image dimensions
df_img = df.reshape(height, width)
line_level_img = line_level.reshape(height, width)
closest_img = closest.reshape(height, width, 2)  # assuming 'closest' has 2 channels
bg_mask_img = bg_mask.reshape(height, width)

print(type(line_level_img))

# use def plot_images(imgs, titles=None, cmaps='gray', dpi=100, size=6, pad=.5):
plot_images([df_img, line_level_img, closest_img[:, :, 0], bg_mask_img], 
            titles=['Distance Function Map', 'Line Level Map', 'Closest Line Map', 'Background Mask'],
            cmaps=['gray', 'gray', 'gray', 'gray'])



NameError: name 'Image' is not defined